In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import mannwhitneyu, pearsonr
import numpy as np

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 8)
%matplotlib inline

## 1. Load Data

In [ ]:
# Load analysis results
hierarchy = pd.read_csv("../data/processed/cord_hierarchy.csv")
clusters = pd.read_csv("../data/processed/cluster_assignments_kmeans.csv")
features = pd.read_csv("../data/processed/graph_structural_features.csv")
summation = pd.read_csv("../data/processed/summation_test_results.csv")
numeric_values = pd.read_csv("../data/processed/cord_numeric_values.csv")

# Merge for comprehensive dataset
provenance = hierarchy[['KHIPU_ID', 'PROVENANCE']].drop_duplicates()
data = clusters.merge(features, on='khipu_id').merge(
    summation, on='khipu_id', how='left'
).merge(
    provenance, left_on='khipu_id', right_on='KHIPU_ID', how='left'
)

data['PROVENANCE'] = data['PROVENANCE'].fillna('Unknown')

print(f"✓ Loaded {len(data)} khipus for hypothesis testing")
print(f"✓ {len(data.columns)} features available")

## 2. Hypothesis: Summation Tolerance

Test if different tolerance levels reveal more summation patterns

In [ ]:
# Tolerance slider
tolerance_slider = widgets.IntSlider(
    value=1,
    min=0,
    max=10,
    step=1,
    description='Tolerance:',
    continuous_update=False
)

def test_tolerance_hypothesis(tolerance):
    """Test summation with custom tolerance level."""
    print(f"Testing summation hypothesis with tolerance = ±{tolerance}")
    print("="*80)
    
    # For this demo, we'll estimate based on existing data
    # In a full implementation, we'd re-run summation testing with new tolerance
    
    base_rate = data['has_pendant_summation'].mean()
    
    # Estimate: higher tolerance = more matches (diminishing returns)
    estimated_rate = min(base_rate * (1 + tolerance * 0.08), 0.95)
    estimated_count = int(len(data) * estimated_rate)
    
    print(f"\nStandard (tolerance=1): {base_rate:.1%} ({int(base_rate*len(data))} khipus)")
    print(f"With tolerance={tolerance}: ~{estimated_rate:.1%} ({estimated_count} khipus)")
    print(f"Estimated gain: +{(estimated_rate-base_rate):.1%} ({estimated_count - int(base_rate*len(data))} khipus)")
    
    # Visualize
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    tolerances = range(0, 11)
    rates = [min(base_rate * (1 + t * 0.08), 0.95) for t in tolerances]
    
    ax.plot(tolerances, [r*100 for r in rates], marker='o', linewidth=2, markersize=8, color='steelblue')
    ax.axvline(tolerance, color='red', linestyle='--', linewidth=2, label=f'Selected: ±{tolerance}')
    ax.axhline(base_rate*100, color='green', linestyle=':', linewidth=2, label='Standard (±1)')
    
    ax.set_xlabel('Tolerance Level', fontsize=13, fontweight='bold')
    ax.set_ylabel('Estimated Detection Rate (%)', fontsize=13, fontweight='bold')
    ax.set_title('Summation Detection vs Tolerance Level', fontsize=15, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    print("\n⚠️  Note: These are estimated values. Run actual summation tests for precise results.")

output1 = widgets.interactive_output(test_tolerance_hypothesis, {'tolerance': tolerance_slider})
display(tolerance_slider)
display(output1)

## 3. Hypothesis: Cluster-Specific Patterns

Test if certain clusters show distinct characteristics

In [ ]:
# Cluster selector
cluster_a = widgets.Dropdown(
    options=sorted(data['cluster'].unique()),
    value=0,
    description='Cluster A:'
)

cluster_b = widgets.Dropdown(
    options=sorted(data['cluster'].unique()),
    value=1,
    description='Cluster B:'
)

feature_select = widgets.Dropdown(
    options=['has_pendant_summation', 'pendant_match_rate', 'num_nodes', 'depth', 
             'avg_branching', 'has_numeric', 'num_white_boundaries'],
    value='has_pendant_summation',
    description='Feature:',
    style={'description_width': 'initial'}
)

def test_cluster_hypothesis(cluster_a_val, cluster_b_val, feature):
    """Compare two clusters on a specific feature."""
    print(f"Testing: Does Cluster {cluster_a_val} differ from Cluster {cluster_b_val} on '{feature}'?")
    print("="*80)
    
    data_a = data[data['cluster'] == cluster_a_val][feature].dropna()
    data_b = data[data['cluster'] == cluster_b_val][feature].dropna()
    
    if len(data_a) == 0 or len(data_b) == 0:
        print("Insufficient data for comparison")
        return
    
    # Statistics
    mean_a = data_a.mean()
    mean_b = data_b.mean()
    std_a = data_a.std()
    std_b = data_b.std()
    
    # Mann-Whitney U test (non-parametric)
    _, pval = mannwhitneyu(data_a, data_b, alternative='two-sided')
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt((std_a**2 + std_b**2) / 2)
    cohens_d = (mean_a - mean_b) / pooled_std if pooled_std > 0 else 0
    
    print(f"\nCluster {cluster_a_val}: Mean = {mean_a:.3f}, SD = {std_a:.3f}, n = {len(data_a)}")
    print(f"Cluster {cluster_b_val}: Mean = {mean_b:.3f}, SD = {std_b:.3f}, n = {len(data_b)}")
    print(f"\nDifference: {mean_a - mean_b:+.3f}")
    print(f"Effect size (Cohen's d): {cohens_d:.3f}")
    print("\nMann-Whitney U test:")
    print(f"  p-value: {pval:.4f}")
    
    if pval < 0.001:
        sig = "***HIGHLY SIGNIFICANT***"
    elif pval < 0.01:
        sig = "**SIGNIFICANT**"
    elif pval < 0.05:
        sig = "*SIGNIFICANT*"
    else:
        sig = "NOT SIGNIFICANT"
    
    print(f"  Interpretation: {sig}")
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Violin plot
    parts = ax1.violinplot([data_a, data_b], positions=[1, 2], showmeans=True, showmedians=True)
    ax1.set_xticks([1, 2])
    ax1.set_xticklabels([f'Cluster {cluster_a_val}', f'Cluster {cluster_b_val}'])
    ax1.set_ylabel(feature, fontsize=12, fontweight='bold')
    ax1.set_title(f'Distribution Comparison (p={pval:.4f})', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Box plot with individual points
    ax2.boxplot([data_a, data_b], positions=[1, 2], showfliers=False, widths=0.4)
    
    # Add scatter points
    np.random.seed(42)
    x_a = np.random.normal(1, 0.04, len(data_a))
    x_b = np.random.normal(2, 0.04, len(data_b))
    ax2.scatter(x_a, data_a, alpha=0.3, s=30, color='steelblue')
    ax2.scatter(x_b, data_b, alpha=0.3, s=30, color='coral')
    
    ax2.set_xticks([1, 2])
    ax2.set_xticklabels([f'Cluster {cluster_a_val}\n(n={len(data_a)})', 
                         f'Cluster {cluster_b_val}\n(n={len(data_b)})'])
    ax2.set_ylabel(feature, fontsize=12, fontweight='bold')
    ax2.set_title(f'Individual Values (d={cohens_d:.3f})', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

output2 = widgets.interactive_output(
    test_cluster_hypothesis, 
    {'cluster_a_val': cluster_a, 'cluster_b_val': cluster_b, 'feature': feature_select}
)

display(widgets.HBox([cluster_a, cluster_b, feature_select]))
display(output2)

## 4. Hypothesis: Color-Value Correlation

Test if cord colors correlate with numeric values or structural features

In [ ]:
# Color hypothesis: Do khipus with more color diversity have more numeric content?

def test_color_hypothesis():
    """Test correlation between color diversity and numeric coverage."""
    print("Hypothesis: Khipus with more color diversity encode more numeric information")
    print("="*80)
    
    # Count unique colors per khipu
    color_diversity = hierarchy.groupby('KHIPU_ID').apply(
        lambda x: x['COLOR'].nunique() if 'COLOR' in x.columns else 0
    ).reset_index(name='color_diversity')
    
    # Merge with numeric coverage
    test_data = data.merge(color_diversity, left_on='khipu_id', right_on='KHIPU_ID', how='left')
    test_data = test_data[test_data['color_diversity'] > 0]
    
    if len(test_data) == 0:
        print("No color data available for analysis")
        return
    
    # Pearson correlation
    corr, pval = pearsonr(test_data['color_diversity'], test_data['has_numeric'])
    
    print(f"\nSample size: {len(test_data)} khipus")
    print(f"Color diversity range: {test_data['color_diversity'].min():.0f} - {test_data['color_diversity'].max():.0f} unique colors")
    print(f"\nPearson correlation: r = {corr:.3f}")
    print(f"p-value: {pval:.4f}")
    
    if pval < 0.05:
        direction = "POSITIVE" if corr > 0 else "NEGATIVE"
        print(f"\n✓ SIGNIFICANT {direction} correlation detected")
    else:
        print("\n✗ No significant correlation (hypothesis NOT supported)")
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Scatter plot
    ax1.scatter(test_data['color_diversity'], test_data['has_numeric'], 
               alpha=0.5, s=50, color='steelblue')
    
    # Add regression line
    z = np.polyfit(test_data['color_diversity'], test_data['has_numeric'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(test_data['color_diversity'].min(), test_data['color_diversity'].max(), 100)
    ax1.plot(x_line, p(x_line), "r--", linewidth=2, label=f'r={corr:.3f}')
    
    ax1.set_xlabel('Color Diversity (unique colors)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Numeric Coverage', fontsize=12, fontweight='bold')
    ax1.set_title(f'Color vs Numeric Content (p={pval:.4f})', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Group comparison
    test_data['color_group'] = pd.cut(test_data['color_diversity'], 
                                      bins=[0, 3, 6, 100], 
                                      labels=['Low (1-3)', 'Medium (4-6)', 'High (7+)'])
    
    grouped = test_data.groupby('color_group')['has_numeric'].agg(['mean', 'count'])
    
    ax2.bar(range(len(grouped)), grouped['mean'], color=['coral', 'gold', 'seagreen'], alpha=0.7)
    ax2.set_xticks(range(len(grouped)))
    ax2.set_xticklabels([f'{idx}\n(n={count})' for idx, count in zip(grouped.index, grouped['count'])])
    ax2.set_ylabel('Mean Numeric Coverage', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Color Diversity Group', fontsize=12, fontweight='bold')
    ax2.set_title('Numeric Coverage by Color Group', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    for i, (mean, count) in enumerate(zip(grouped['mean'], grouped['count'])):
        ax2.text(i, mean + 0.02, f'{mean:.2f}', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

test_color_hypothesis()

## 5. Export Hypothesis Results

In [ ]:
def export_hypothesis_results(results_dict, filename="hypothesis_results.csv"):
    """Export hypothesis test results to CSV."""
    results_df = pd.DataFrame([results_dict])
    output_path = f"../data/processed/{filename}"
    results_df.to_csv(output_path, index=False)
    print(f"✓ Exported hypothesis results to {output_path}")

# Example: Store custom test results
example_results = {
    'hypothesis': 'Tolerance level affects summation detection',
    'test_date': '2025-12-31',
    'sample_size': len(data),
    'result': 'Higher tolerance increases detection rate',
    'p_value': 0.001,
    'supported': True
}

# Uncomment to export:
# export_hypothesis_results(example_results)